In [1]:
import requests
import time 
import json
import os
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def get_date_data(country, brand , storename):
    response = requests.get(f'https://ana-slo.com/ホールデータ/{country}/{brand}{storename}-データ一覧/')
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('div' , id= 'table')

    #找到所有有 a 的日期
    a = table.find_all('a')
    a_list = []
    for date in a:
        a_list.append(date.text.strip()[:10].replace('/', '-'))

    return a_list

In [4]:
#the main function
def get_slot_data(date , brand , storename):
    response = requests.get(f'https://ana-slo.com/{date}-{brand}{storename}-data/')
    soup = BeautifulSoup(response.text, 'html.parser')

    div = soup.find_all('div', id='all_data_block')
    tr = div[0].find_all('tr')
    columns_slot = tr[0].text.strip().split('\n')

    data_list = []
    for td in tr[1:len(tr)]:
        data = td.text.strip().split('\n')
        if len(data)==9:
            data_list.append(data)

    df = pd.DataFrame(data_list, columns=columns_slot)
    #新增日期欄位為date轉為日期格式
    df['date'] = pd.to_datetime(date)
    return df
for d in a_list[:14]:
    try :

        df = get_slot_data(date =d , brand=  'マルハン' , storename= '石和店')
        #每個df concat起來
        if d == a_list[0]:
            df_all = df
        else:
            df_all = pd.concat([df_all, df], axis=0)
    except exception as e:
        print(e)
        print(f'{d} is not available')

#------------------------------------------------
#用差枚總數排行
df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
top10_coins = df_all.groupby('台番号')['差枚'].sum().sort_values(ascending=False).head(10)
df_all['win'] = df_all['差枚'].apply(lambda x: 1 if x > 0 else 0)
#top10.index 篩選df_all['台號'] == top10.index
df_top10_coins = df_all[df_all['台番号'].isin(top10_coins.index)]
#計算top10的WIN RATE
top10_coins_win_rate = df_top10_coins.groupby('台番号')['win'].mean().sort_values(ascending=False).head(10)
df_final_coins = pd.merge(top10_coins, top10_coins_win_rate, on='台番号', how='left')
print(df_final_coins)

#------------------------------------------------
#計算['win']裡的1的比例 groupby 台號
win_rate_top10  = df_all.groupby('台番号')['win'].mean().sort_values(ascending=False).head(10)
#merge對應的差枚
top10_win_rate_coins = df_all[df_all['台番号'].isin(win_rate_top10.index)]
top10_win_rate_coins = top10_win_rate_coins.groupby('台番号')['差枚'].sum()
df_final_win_rate = pd.merge( win_rate_top10,top10_win_rate_coins , on='台番号', how='left')
print(df_final_win_rate)

以下分析

In [ ]:
df_all['差枚'] = df_all['差枚'].str.replace(',', '').str.replace('+', '').astype(int)
#差枚欄位改為int
df_all['差枚'].astype(int)

In [ ]:
#小丑
dic = {1 : 1/163.8 ,
2 : 1/159.1
,3 : 1/148.6
,4 : 1/135.2
,5 : 1/126.8
,6 : 1/114.6}

In [ ]:
dic.values()

In [ ]:
def safe_division(x):
    try:
        return eval(x) if '/' in x else float(x)
    except ZeroDivisionError:
        return 0  # 或者其他你希望在分母為零時返回的值

df_all['合成確率'] = df_all['合成確率'].apply(safe_division)
df_all['合成確率'].astype(float)

In [ ]:
min(dic.values(), key=lambda x:abs(x-0.006))

In [ ]:
#最接近dic 的value給予其key 例如最接近 1/163.8 的值為1 , 1/159.1 的值為2
def find_nearest(dic , value):
    nearest = min(dic.values(), key=lambda x:abs(x-value))
    for k, v in dic.items():
        if v == nearest:
            return k


In [ ]:
#新增一個欄位並套用find_nearest function
df_all['合成確率_rank'] = df_all['合成確率'].apply(lambda x : find_nearest(dic , x))

In [ ]:
マイジャグラーV = df_all[df_all['機種名']=='マイジャグラーV'].sort_index(ascending=True)

In [ ]:
マイジャグラーV.to_csv('マイジャグラーV.csv', index=False)

In [ ]:
df_all.to_csv('slot_data_マルハン石和店_20240514.csv', index=False)

In [ ]:
#test for the data image

respinse = requests.get('https://ana-slo.com/2024-05-07-%e3%83%9e%e3%83%ab%e3%83%8f%e3%83%b3%e7%9f%b3%e5%92%8c%e5%ba%97-data/')
soup = BeautifulSoup(respinse.text, 'html.parser')

div = soup.find_all('div', id='tab02_0')
#找到div裡的script  
script = div[0].find_all('script')
len(script)
#提取script裡的 x_value , y_value
x_value = script[0].text.split('x_value = ')[1].split(';')[0]
y_value = script[0].text.split('y_value = ')[1].split(';')[0]
#用x_value , y_value 劃出折線圖
import matplotlib.pyplot as plt
x = json.loads(x_value)
y = json.loads(y_value)
plt.plot(x, y)
plt.show()

